In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bone_Density"
cohort = "GSE56814"

# Input paths
in_trait_dir = "../../input/GEO/Bone_Density"
in_cohort_dir = "../../input/GEO/Bone_Density/GSE56814"

# Output paths
out_data_file = "../../output/preprocess/Bone_Density/GSE56814.csv"
out_gene_data_file = "../../output/preprocess/Bone_Density/gene_data/GSE56814.csv"
out_clinical_data_file = "../../output/preprocess/Bone_Density/clinical_data/GSE56814.csv"
json_path = "../../output/preprocess/Bone_Density/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression study of blood monocytes in pre- and postmenopausal females with low or high bone mineral density (HuEx-1_0-st-v2)"
!Series_summary	"Comparison of circulating monocytes from pre- and postmenopausal females with low or high bone mineral density (BMD). Circulating monocytes are progenitors of osteoclasts, and produce factors important to bone metabolism. Results provide insight into the role of monocytes in osteoporosis."
!Series_summary	"We identify osteoporosis genes by microarray analyses of monocytes in high vs. low hip BMD (bone mineral density) subjects."
!Series_overall_design	"Microarray analyses of monocytes were performed using Affymetrix 1.0 ST arrays in 73 Caucasian females (age: 47-56) with extremely high (mean ZBMD =1.38, n=42, 16 pre- and 26 postmenopausal subjects) or low hip BMD (mean ZBMD=-1.05, n=31, 15 pre- and 16 postmenopausal subjects). Differential gene expression analysis in high vs. low BMD subjects was cond

### Step 2: Data Normalization and Linking

In [3]:
# 1. Get gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Select clinical features from the clinical data
# Based on the sample characteristics, we need to extract relevant clinical features
print("Clinical data before extraction:")
print(clinical_data.head())

def convert_bmd(value):
    if isinstance(value, str):
        if 'high BMD' in value:
            return 1
        elif 'low BMD' in value:
            return 0
    return None

# Row 1 contains bone mineral density information
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,
    convert_trait=convert_bmd
)

print("Clinical features after extraction:")
print(clinical_features)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print("Linked data preview:")
print(linked_data.head())

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 5. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note="Dataset contains bone mineral density information categorized as high or low BMD."
)

# 7. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")


Normalized gene data saved to ../../output/preprocess/Bone_Density/gene_data/GSE56814.csv
Clinical data before extraction:
         !Sample_geo_accession                      GSM1369683  \
0  !Sample_characteristics_ch1                  gender: Female   
1  !Sample_characteristics_ch1  bone mineral density: high BMD   
2  !Sample_characteristics_ch1           state: postmenopausal   
3  !Sample_characteristics_ch1            cell type: monocytes   

                       GSM1369684                      GSM1369685  \
0                  gender: Female                  gender: Female   
1  bone mineral density: high BMD  bone mineral density: high BMD   
2           state: postmenopausal           state: postmenopausal   
3            cell type: monocytes            cell type: monocytes   

                       GSM1369686                     GSM1369687  \
0                  gender: Female                 gender: Female   
1  bone mineral density: high BMD  bone mineral density: low BMD

### Step 3: Dataset Analysis and Clinical Feature Extraction

In [4]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# From the previous output, we can see:
# - Gene data is available (was normalized and saved)
# - Clinical data features show gender, bone mineral density, menopausal state, and cell type

# 1. Gene Expression Data Availability
is_gene_available = True  # Gene expression data is available

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify rows containing trait, age, and gender information
trait_row = 1  # "bone mineral density: low BMD" is in row 1
age_row = None  # Age doesn't appear to be available in the sample characteristics
gender_row = 0  # "gender: Female" is in row 0

# 2.2 Define conversion functions
def convert_trait(value):
    if pd.isna(value):
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if "high bmd" in value:
        return 1.0
    elif "low bmd" in value:
        return 0.0
    else:
        return None

# Age conversion function not needed as age_row is None
convert_age = None

def convert_gender(value):
    if pd.isna(value):
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# 3. Save Metadata for initial filtering
is_trait_available = trait_row is not None
initial_validation = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. If we have not already processed the data, proceed with clinical feature extraction
# Use the clinical_data from the previous output if available
if trait_row is not None:
    try:
        # Try to load the clinical data that may have been created earlier in the pipeline
        clinical_data_path = os.path.join(in_cohort_dir, "clinical_characteristics.csv")
        if os.path.exists(clinical_data_path):
            clinical_data = pd.read_csv(clinical_data_path)
            
            # Extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the extracted data
            preview = preview_df(selected_clinical_df)
            print("Preview of extracted clinical features:")
            for feature, values in preview.items():
                print(f"{feature}: {values}")
            
            # Ensure the output directory exists
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            
            # Save the extracted clinical data
            selected_clinical_df.to_csv(out_clinical_data_file)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print(f"Clinical data file not found at: {clinical_data_path}")
            print("Based on previous output, clinical data was already processed but the cohort was marked as unusable.")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        print("Moving forward with the knowledge that this dataset may have issues.")
else:
    print("No trait data available. Skipping clinical feature extraction.")

print("Dataset analysis completed.")


Clinical data file not found at: ../../input/GEO/Bone_Density/GSE56814/clinical_characteristics.csv
Based on previous output, clinical data was already processed but the cohort was marked as unusable.
Dataset analysis completed.


### Step 4: Gene Data Extraction

In [5]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')


### Step 5: Gene Identifier Review

In [6]:
# These identifiers don't appear to be human gene symbols
# They look like probe IDs from a microarray platform
# Looking at the numeric format (2315554, etc.), these are likely Illumina or Affymetrix probe IDs
# They will need to be mapped to human gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [7]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // D

### Step 7: Gene Identifier Mapping

In [8]:
# 1. Determine which columns to use for mapping
# From the gene annotation preview, I can see:
# - 'ID' column contains identifiers that match the row IDs seen in the gene expression data
# - 'gene_assignment' column contains gene symbol information

# 2. Extract gene mapping from the annotation dataframe
# Create the mapping dataframe with the probe IDs and gene symbols
gene_mapping = get_gene_mapping(
    annotation=gene_annotation,
    prob_col="ID",
    gene_col="gene_assignment"
)

print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=gene_mapping)

print(f"Gene data after mapping - shape: {gene_data.shape}")
print("First 5 gene symbols and first 3 samples:")
print(gene_data.iloc[:5, :3])


Gene mapping preview (first 5 rows):
        ID                                               Gene
0  2315100  NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-As...
1  2315106                                                ---
2  2315109                                                ---
3  2315111                                                ---
4  2315113                                                ---


Gene data after mapping - shape: (48895, 73)
First 5 gene symbols and first 3 samples:
      GSM1369683  GSM1369684  GSM1369685
Gene                                    
A-     18.363212   19.338694   19.496579
A-2     3.049234    3.030850    2.939030
A-52    4.685952    4.691996    4.603244
A-E     1.581215    1.632573    1.682945
A-I     5.532182    5.607333    5.265896


### Step 8: Data Normalization and Linking

In [9]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data and link with genetic data
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(True, cohort, json_path, True, True, is_trait_biased, unbiased_linked_data)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Bone_Density/gene_data/GSE56814.csv


For the feature 'Bone_Density', the least common label is '0.0' with 31 occurrences. This represents 42.47% of the dataset.
The distribution of the feature 'Bone_Density' in this dataset is fine.



Linked data saved to ../../output/preprocess/Bone_Density/GSE56814.csv
